In [1]:
import os
import numpy as np
import time
from PIL import Image
import utils

from tqdm import notebook
def tqdm(x, **kargs):
    return notebook.tqdm(x, leave=False, **kargs)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms
import torchvision.models as models

from efficientnet_pytorch import EfficientNet

from sklearn.metrics import f1_score, precision_recall_fscore_support

In [2]:
class customDataset(Dataset):
    def __init__(self, datatype, transform):
        ##############################################
        ### Initialize paths, transforms, and so on
        ##############################################
        self.transform = transform

        filename = open('./crawling/{}_list'.format(datatype), 'r')
        info = filename.readlines()
        self.images = [row.split(',')[0] for row in info]
        self.labels = [int(row.split(',')[1].strip()) for row in info]
        
        assert len(self.images) == len(self.labels), 'mismatched length!'
        print("image shape: {}, label shape: {}".format(len(self.images), len(self.labels)))
        
        
    def __getitem__(self, index):
        ##############################################
        # 1. Read from file (using numpy.fromfile, PIL.Image.open)
        # 2. Preprocess the data (torchvision.Transform).
        # 3. Return the data (e.g. image and label)
        ##############################################
        
        image = Image.open(self.images[index]).convert('RGB')
        label = self.labels[index]
        
        if self.transform is not None:
            image = self.transform(image)
        return image, label
        
    def __len__(self):
        ##############################################
        ### Indicate the total size of the dataset
        ##############################################
        return len(self.images)

In [3]:
batch_size = 32
num_workers = 10
# min (100, 100), max (5792, 8688)
resize_size = (224, 224)
MODEL = 'efficientnet-b4'
# MODEL = 'CUSTOM'

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.5, contrast=0.5),
        # transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ]),
    'test': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ]),
}

In [5]:
trainset = customDataset(datatype='train',
                         transform=data_transforms['train'])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=num_workers, shuffle=True)

image shape: 46078, label shape: 46078


In [6]:
testset = customDataset(datatype='test',
                        transform=data_transforms['test'])
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

image shape: 11519, label shape: 11519


In [7]:
# for imgs, lbls in trainloader:
#     print('Size of image:', imgs.size())  
#     print('Type of image:', imgs.dtype)
#     print('Size of label:', lbls.size())  
#     print('Type of label:', lbls.dtype)
#     break

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
net = utils.Net()
net.to(device)

Loaded pretrained weights for efficientnet-b4


rnel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          28, 672, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticSamePadding(
          672, 160, kernel_size=(1, 1), stride=(1, 1), bias=False
          (static_padding): Identity()
        )
        (_bn2): BatchNorm2d(160, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_swish): MemoryEfficientSwish()
      )
      (17): MBConvBlock(
        (_expand_conv): Conv2dStaticSamePadding(
          160, 960, kernel_size=(1, 1), stride=(1, 1), bias=False
          (static_padding): Identity()
        )
        (_bn0): BatchNorm2d(960, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_depthwise_conv): Conv2dStaticSamePadding(
          960, 960, kernel_size=(5, 5), stride=(1, 1), groups=960, bias=False
         

In [10]:
# positive: 15433, negative: 41118
# positive: 17827, negative: 39770
class_weights = torch.FloatTensor([1, 2.309]).cuda()
# class_weights = torch.FloatTensor([1, 3]).cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights)
# criterion = nn.CrossEntropyLoss()
epochs = 5
size = 46078 // batch_size
timestr = time.strftime("%Y%m%d-%H%M%S")

In [11]:
### First
LR = 1e-2
# optimizer = optim.Adam(net.parameters(), lr=1e-3)
optimizer = optim.SGD(net.parameters(), lr=LR)

for epoch in tqdm(range(epochs), desc='Epochs: '):  # loop over the dataset multiple times
    print("================== EPOCH INFO: {} ==================".format(epoch + 1))
    net.train()     # Train mode
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, desc='Progress:'), 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        labels = labels.view(-1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        # print(data[0].shape, data[1].shape)
        # print(inputs.shape)
        # print(labels.shape, outputs.shape)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        interval = 500
        if i % interval == 0 and i != 0:    # print every interval
            _, y_pred_tag = torch.max(outputs, 1)
            correct_results_sum = (y_pred_tag == labels).sum().float()
            acc = correct_results_sum / labels.shape[0]

            y_pred_np = y_pred_tag.cpu().detach().numpy()
            y_true_np = labels.cpu().detach().numpy()
            print("==============================================")
            print("Batch acc = {} ({} / {})".format(acc, correct_results_sum, labels.shape[0]))
            print("Batch f1-score = {}".format(f1_score(y_true_np, y_pred_np, average='binary')))
            print(precision_recall_fscore_support(y_true_np, y_pred_np, average='binary'))

            print("Prediction distribution:")
            print(np.unique(y_pred_np, return_counts=True), np.unique(y_true_np, return_counts=True))
            # print(y_pred_np, y_true_np)

            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / interval))
            running_loss = 0.0

    # Eval mode
    net.eval()
    # Training score
    y_pred_train = np.array([])
    y_true_train = np.array([])
    with torch.no_grad():
        for data_test in tqdm(trainloader):
            images, labels = data_test[0].to(device), data_test[1].to(device)
            outputs = net(images)

            _, y_pred_tag = torch.max(outputs, 1)
            y_pred_train = np.hstack([y_pred_train, y_pred_tag.cpu().detach().numpy()])

            y_true_train = np.hstack([y_true_train, labels.cpu().detach().numpy()])

    precision_train, recall_train, f1_train, support_train = precision_recall_fscore_support(y_true_train, y_pred_train, average='binary')

    print()
    print("Training prediction:")
    print(np.unique(y_pred_train, return_counts=True), np.unique(y_true_train, return_counts=True))
    print(y_pred_train[0:10], y_true_train[0:10])
    print("F1 of model on train set: ", f1_score(y_true_train, y_pred_train, average='binary'))
    print("Precision: {}, Recall: {}, F1: {}".format(precision_train, recall_train, f1_train))

    # Testing score
    y_pred_test = np.array([])
    y_true_test = np.array([])
    with torch.no_grad():
        for data_test in tqdm(testloader):
            images, labels = data_test[0].to(device), data_test[1].to(device)
            outputs = net(images)

            _, y_pred_tag = torch.max(outputs, 1)
            y_pred_test = np.hstack([y_pred_test, y_pred_tag.cpu().detach().numpy()])
                
            y_true_test = np.hstack([y_true_test, labels.cpu().detach().numpy()])
    
    precision_test, recall_test, f1_test, support_test = precision_recall_fscore_support(y_true_test, y_pred_test, average='binary')

    print("Testing prediction:")
    print(np.unique(y_pred_test, return_counts=True), np.unique(y_true_test, return_counts=True))
    print(y_pred_test[0:10], y_true_test[0:10])
    print("F1 of my model on test set: ", f1_score(y_true_test, y_pred_test, average='binary'))
    print("Precision: {}, Recall: {}, F1: {}".format(precision_test, recall_test, f1_test))


    # record precision, recall, f1-score into file
    with open('record/{}_{}_train_loss'.format(MODEL, timestr), 'a') as f_train, open('record/{}_{}_test_loss'.format(MODEL, timestr), 'a') as f_test:
        f_train.write(str(precision_train) + ', ' + str(recall_train) + ', ' + str(f1_train) + '\n')
        f_test.write(str(precision_test) + ', ' + str(recall_test) + ', ' + str(f1_test) + '\n')

    # save model for every epoch
    torch.save(net.state_dict(), './model/{}_{}_{}_{}_{}_{}'.format(MODEL, timestr, LR, epoch, f1_train, f1_test))

================== EPOCH INFO: 1 ==================


Batch acc = 0.5 (16.0 / 32)
Batch f1-score = 0.19999999999999998
(0.18181818181818182, 0.2222222222222222, 0.19999999999999998, None)
Prediction distribution:
(array([0, 1]), array([21, 11])) (array([0, 1]), array([23,  9]))
[1,   501] loss: 0.731
Batch acc = 0.625 (20.0 / 32)
Batch f1-score = 0.45454545454545453
(0.4166666666666667, 0.5, 0.45454545454545453, None)
Prediction distribution:
(array([0, 1]), array([20, 12])) (array([0, 1]), array([22, 10]))
[1,  1001] loss: 0.724



Training prediction:
(array([0., 1.]), array([14619, 31459])) (array([0., 1.]), array([31816, 14262]))
[1. 1. 0. 1. 0. 1. 0. 0. 0. 1.] [1. 0. 1. 0. 0. 1. 0. 0. 0. 1.]
F1 of model on train set:  0.45108374707464843
Precision: 0.3277917289170031, Recall: 0.7230402468097041, F1: 0.45108374707464843


Testing prediction:
(array([0., 1.]), array([3465, 8054])) (array([0., 1.]), array([7954, 3565]))
[1. 0. 1. 1. 1. 0. 1. 0. 1. 1.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
F1 of my model on test set:  0.44857560891643006
Precision: 0.3235659299726844, Recall: 0.7309957924263675, F1: 0.44857560891643006
================== EPOCH INFO: 2 ==================


Batch acc = 0.34375 (11.0 / 32)
Batch f1-score = 0.22222222222222224
(0.14285714285714285, 0.5, 0.22222222222222224, None)
Prediction distribution:
(array([0, 1]), array([11, 21])) (array([0, 1]), array([26,  6]))
[2,   501] loss: 0.719
Batch acc = 0.65625 (21.0 / 32)
Batch f1-score = 0.15384615384615383
(0.14285714285714285, 0.16666666666666666, 0.15384615384615383, None)
Prediction distribution:
(array([0, 1]), array([25,  7])) (array([0, 1]), array([26,  6]))
[2,  1001] loss: 0.721


...


Training prediction:
(array([0., 1.]), array([32782, 13296])) (array([0., 1.]), array([31816, 14262]))
[0. 0. 1. 1. 0. 1. 0. 1. 0. 0.] [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
F1 of model on train set:  0.33471224326874227
Precision: 0.34687123947051746, Recall: 0.3233768054971252, F1: 0.33471224326874227


Testing prediction:
(array([0., 1.]), array([8180, 3339])) (array([0., 1.]), array([7954, 3565]))
[1. 0. 0. 0. 0. 0. 0. 0. 1. 0.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
F1 of my model on test set:  0.3253186558516802
Precision: 0.3363282419886193, Recall: 0.3150070126227209, F1: 0.3253186558516802
================== EPOCH INFO: 5 ==================


Batch acc = 0.5625 (18.0 / 32)
Batch f1-score = 0.5333333333333333
(0.4, 0.8, 0.5333333333333333, None)
Prediction distribution:
(array([0, 1]), array([12, 20])) (array([0, 1]), array([22, 10]))
[5,   501] loss: 0.719
Batch acc = 0.5 (16.0 / 32)
Batch f1-score = 0.2727272727272727
(0.21428571428571427, 0.375, 0.2727272727272727, None)
Prediction distribution:
(array([0, 1]), array([18, 14])) (array([0, 1]), array([24,  8]))
[5,  1001] loss: 0.721



Training prediction:
(array([0., 1.]), array([39109,  6969])) (array([0., 1.]), array([31816, 14262]))
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] [0. 0. 1. 0. 0. 0. 0. 1. 1. 0.]
F1 of model on train set:  0.23663510903866986
Precision: 0.3604534366480126, Recall: 0.17613237975038565, F1: 0.23663510903866986


Testing prediction:
(array([0., 1.]), array([9867, 1652])) (array([0., 1.]), array([7954, 3565]))
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
F1 of my model on test set:  0.22656699252443932
Precision: 0.3577481840193705, Recall: 0.16577840112201964, F1: 0.22656699252443932


In [12]:
### Second try
LR = 1e-3
# optimizer = optim.Adam(net.parameters(), lr=1e-3)
optimizer = optim.SGD(net.parameters(), lr=LR)

for epoch in tqdm(range(epochs), desc='Epochs: '):  # loop over the dataset multiple times
    print("================== EPOCH INFO: {} ==================".format(epoch + 1))
    net.train()     # Train mode
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, desc='Progress:'), 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        labels = labels.view(-1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        # print(data[0].shape, data[1].shape)
        # print(inputs.shape)
        # print(labels.shape, outputs.shape)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        interval = 500
        if i % interval == 0 and i != 0:    # print every interval
            _, y_pred_tag = torch.max(outputs, 1)
            correct_results_sum = (y_pred_tag == labels).sum().float()
            acc = correct_results_sum / labels.shape[0]

            y_pred_np = y_pred_tag.cpu().detach().numpy()
            y_true_np = labels.cpu().detach().numpy()
            print("==============================================")
            print("Batch acc = {} ({} / {})".format(acc, correct_results_sum, labels.shape[0]))
            print("Batch f1-score = {}".format(f1_score(y_true_np, y_pred_np, average='binary')))
            print(precision_recall_fscore_support(y_true_np, y_pred_np, average='binary'))

            print("Prediction distribution:")
            print(np.unique(y_pred_np, return_counts=True), np.unique(y_true_np, return_counts=True))
            # print(y_pred_np, y_true_np)

            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / interval))
            running_loss = 0.0

    # Eval mode
    net.eval()
    # Training score
    y_pred_train = np.array([])
    y_true_train = np.array([])
    with torch.no_grad():
        for data_test in tqdm(trainloader):
            images, labels = data_test[0].to(device), data_test[1].to(device)
            outputs = net(images)

            _, y_pred_tag = torch.max(outputs, 1)
            y_pred_train = np.hstack([y_pred_train, y_pred_tag.cpu().detach().numpy()])

            y_true_train = np.hstack([y_true_train, labels.cpu().detach().numpy()])

    precision_train, recall_train, f1_train, support_train = precision_recall_fscore_support(y_true_train, y_pred_train, average='binary')

    print()
    print("Training prediction:")
    print(np.unique(y_pred_train, return_counts=True), np.unique(y_true_train, return_counts=True))
    print(y_pred_train[0:10], y_true_train[0:10])
    print("F1 of model on train set: ", f1_score(y_true_train, y_pred_train, average='binary'))
    print("Precision: {}, Recall: {}, F1: {}".format(precision_train, recall_train, f1_train))

    # Testing score
    y_pred_test = np.array([])
    y_true_test = np.array([])
    with torch.no_grad():
        for data_test in tqdm(testloader):
            images, labels = data_test[0].to(device), data_test[1].to(device)
            outputs = net(images)

            _, y_pred_tag = torch.max(outputs, 1)
            y_pred_test = np.hstack([y_pred_test, y_pred_tag.cpu().detach().numpy()])
                
            y_true_test = np.hstack([y_true_test, labels.cpu().detach().numpy()])
    
    precision_test, recall_test, f1_test, support_test = precision_recall_fscore_support(y_true_test, y_pred_test, average='binary')

    print("Testing prediction:")
    print(np.unique(y_pred_test, return_counts=True), np.unique(y_true_test, return_counts=True))
    print(y_pred_test[0:10], y_true_test[0:10])
    print("F1 of my model on test set: ", f1_score(y_true_test, y_pred_test, average='binary'))
    print("Precision: {}, Recall: {}, F1: {}".format(precision_test, recall_test, f1_test))


    # record precision, recall, f1-score into file
    with open('record/{}_{}_train_loss'.format(MODEL, timestr), 'a') as f_train, open('record/{}_{}_test_loss'.format(MODEL, timestr), 'a') as f_test:
        f_train.write(str(precision_train) + ', ' + str(recall_train) + ', ' + str(f1_train) + '\n')
        f_test.write(str(precision_test) + ', ' + str(recall_test) + ', ' + str(f1_test) + '\n')

    # save model for every epoch
    torch.save(net.state_dict(), './model/{}_{}_{}_{}_{}_{}'.format(MODEL, timestr, LR, epoch, f1_train, f1_test))

================== EPOCH INFO: 1 ==================


Batch acc = 0.4375 (14.0 / 32)
Batch f1-score = 0.4375
(0.3684210526315789, 0.5384615384615384, 0.4375, None)
Prediction distribution:
(array([0, 1]), array([13, 19])) (array([0, 1]), array([19, 13]))
[1,   501] loss: 0.694
Batch acc = 0.3125 (10.0 / 32)
Batch f1-score = 0.26666666666666666
(0.18181818181818182, 0.5, 0.26666666666666666, None)
Prediction distribution:
(array([0, 1]), array([10, 22])) (array([0, 1]), array([24,  8]))
[1,  1001] loss: 0.692



Training prediction:
(array([0., 1.]), array([17053, 29025])) (array([0., 1.]), array([31816, 14262]))
[1. 1. 1. 0. 0. 0. 0. 1. 1. 0.] [0. 0. 0. 1. 0. 0. 1. 0. 0. 0.]
F1 of model on train set:  0.45149814031926444
Precision: 0.33667527993109386, Recall: 0.6851773944748282, F1: 0.45149814031926444


Testing prediction:
(array([0., 1.]), array([4227, 7292])) (array([0., 1.]), array([7954, 3565]))
[1. 1. 1. 1. 1. 0. 0. 0. 0. 1.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
F1 of my model on test set:  0.4434005710601455
Precision: 0.33008776741634666, Recall: 0.6751753155680225, F1: 0.4434005710601455
================== EPOCH INFO: 2 ==================


Batch acc = 0.5 (16.0 / 32)
Batch f1-score = 0.3846153846153846
(0.2631578947368421, 0.7142857142857143, 0.3846153846153846, None)
Prediction distribution:
(array([0, 1]), array([13, 19])) (array([0, 1]), array([25,  7]))
[2,   501] loss: 0.693
Batch acc = 0.40625 (13.0 / 32)
Batch f1-score = 0.17391304347826086
(0.14285714285714285, 0.2222222222222222, 0.17391304347826086, None)
Prediction distribution:
(array([0, 1]), array([18, 14])) (array([0, 1]), array([23,  9]))
[2,  1001] loss: 0.692


...


Training prediction:
(array([0., 1.]), array([30644, 15434])) (array([0., 1.]), array([31816, 14262]))
[1. 0. 1. 1. 1. 1. 1. 0. 0. 0.] [0. 0. 0. 1. 1. 0. 0. 0. 1. 1.]
F1 of model on train set:  0.3752693965517242
Precision: 0.36102112219774524, Recall: 0.39068854298134903, F1: 0.3752693965517242


Testing prediction:
(array([0., 1.]), array([7413, 4106])) (array([0., 1.]), array([7954, 3565]))
[1. 0. 0. 0. 1. 0. 0. 0. 0. 0.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
F1 of my model on test set:  0.3788293573197758
Precision: 0.35387238188017534, Recall: 0.40757363253856943, F1: 0.3788293573197758
================== EPOCH INFO: 5 ==================


Batch acc = 0.5625 (18.0 / 32)
Batch f1-score = 0.4166666666666667
(0.3333333333333333, 0.5555555555555556, 0.4166666666666667, None)
Prediction distribution:
(array([0, 1]), array([17, 15])) (array([0, 1]), array([23,  9]))
[5,   501] loss: 0.691
Batch acc = 0.5 (16.0 / 32)
Batch f1-score = 0.3333333333333333
(0.23529411764705882, 0.5714285714285714, 0.3333333333333333, None)
Prediction distribution:
(array([0, 1]), array([15, 17])) (array([0, 1]), array([25,  7]))
[5,  1001] loss: 0.690



Training prediction:
(array([0., 1.]), array([21717, 24361])) (array([0., 1.]), array([31816, 14262]))
[1. 0. 0. 0. 0. 0. 0. 0. 1. 1.] [0. 0. 0. 1. 0. 0. 0. 1. 0. 1.]
F1 of model on train set:  0.43766667529710274
Precision: 0.3469479906407783, Recall: 0.5926237554340205, F1: 0.43766667529710274


Testing prediction:
(array([0., 1.]), array([5548, 5971])) (array([0., 1.]), array([7954, 3565]))
[1. 1. 0. 1. 1. 0. 0. 0. 0. 1.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
F1 of my model on test set:  0.4236577181208054
Precision: 0.3383017919946408, Recall: 0.5666199158485273, F1: 0.4236577181208054


In [15]:
PATH = './model/efficientnet-b4_20201011-232220_0.001_0_0.45149814031926444_0.4434005710601455'
new_net = utils.Net()
new_net.load_state_dict(torch.load(PATH))
new_net.to(device)
new_net.eval()

Loaded pretrained weights for efficientnet-b4


rnel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          28, 672, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticSamePadding(
          672, 160, kernel_size=(1, 1), stride=(1, 1), bias=False
          (static_padding): Identity()
        )
        (_bn2): BatchNorm2d(160, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_swish): MemoryEfficientSwish()
      )
      (17): MBConvBlock(
        (_expand_conv): Conv2dStaticSamePadding(
          160, 960, kernel_size=(1, 1), stride=(1, 1), bias=False
          (static_padding): Identity()
        )
        (_bn0): BatchNorm2d(960, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_depthwise_conv): Conv2dStaticSamePadding(
          960, 960, kernel_size=(5, 5), stride=(1, 1), groups=960, bias=False
         

In [16]:
# Training score
y_pred_train = np.array([])
y_true_train = np.array([])
with torch.no_grad():
    for data_test in tqdm(trainloader):
        images, labels = data_test[0].to(device), data_test[1].to(device)
        outputs = new_net(images)

        _, y_pred_tag = torch.max(outputs, 1)
        y_pred_train = np.hstack([y_pred_train, y_pred_tag.cpu().detach().numpy()])

        y_true_train = np.hstack([y_true_train, labels.cpu().detach().numpy()])

precision_train, recall_train, f1_train, support_train = precision_recall_fscore_support(y_true_train, y_pred_train, average='binary')

print()
print("Training prediction:")
print(np.unique(y_pred_train, return_counts=True), np.unique(y_true_train, return_counts=True))
print(y_pred_train[0:10], y_true_train[0:10])
print("F1 of model on train set: ", f1_score(y_true_train, y_pred_train, average='binary'))
print("Precision: {}, Recall: {}, F1: {}".format(precision_train, recall_train, f1_train))


Training prediction:
(array([0., 1.]), array([14580, 31498])) (array([0., 1.]), array([31816, 14262]))
[1. 1. 1. 1. 1. 0. 1. 1. 1. 0.] [1. 0. 1. 1. 0. 0. 1. 1. 0. 0.]
F1 of model on train set:  0.4508741258741259
Precision: 0.3275128579592355, Recall: 0.723320712382555, F1: 0.4508741258741259


In [17]:
# Testing score
y_pred_test = np.array([])
y_true_test = np.array([])
with torch.no_grad():
    for data_test in tqdm(testloader):
        images, labels = data_test[0].to(device), data_test[1].to(device)
        outputs = new_net(images)

        _, y_pred_tag = torch.max(outputs, 1)
        y_pred_test = np.hstack([y_pred_test, y_pred_tag.cpu().detach().numpy()])
            
        y_true_test = np.hstack([y_true_test, labels.cpu().detach().numpy()])

precision_test, recall_test, f1_test, support_test = precision_recall_fscore_support(y_true_test, y_pred_test, average='binary')

print("Testing prediction:")
print(np.unique(y_pred_test, return_counts=True), np.unique(y_true_test, return_counts=True))
print(y_pred_test[0:10], y_true_test[0:10])
print("F1 of my model on test set: ", f1_score(y_true_test, y_pred_test, average='binary'))
print("Precision: {}, Recall: {}, F1: {}".format(precision_test, recall_test, f1_test))

Testing prediction:
(array([0., 1.]), array([3465, 8054])) (array([0., 1.]), array([7954, 3565]))
[1. 0. 1. 1. 1. 0. 1. 0. 1. 1.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
F1 of my model on test set:  0.44857560891643006
Precision: 0.3235659299726844, Recall: 0.7309957924263675, F1: 0.44857560891643006


In [2]:
new_net = utils.Net()

)
  (15): MBConvBlock(
    (_expand_conv): Conv2dStaticSamePadding(
      112, 672, kernel_size=(1, 1), stride=(1, 1), bias=False
      (static_padding): Identity()
    )
    (_bn0): BatchNorm2d(672, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_depthwise_conv): Conv2dStaticSamePadding(
      672, 672, kernel_size=(3, 3), stride=(1, 1), groups=672, bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn1): BatchNorm2d(672, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_se_reduce): Conv2dStaticSamePadding(
      672, 28, kernel_size=(1, 1), stride=(1, 1)
      (static_padding): Identity()
    )
    (_se_expand): Conv2dStaticSamePadding(
      28, 672, kernel_size=(1, 1), stride=(1, 1)
      (static_padding): Identity()
    )
    (_project_conv): Conv2dStaticSamePadding(
      672, 112, kernel_size=(1, 1), stride=(1, 1), bias=False
      (static_padding): Identity()
    )

In [3]:
new_net.layer_cnt

9